In [127]:
import os
import pandas as pd

In [139]:
data = ["CZ_ALBUMS_TOP_100_mesicni.csv","CZ_SINGLES_DIGITAL_TOP_50__CZ_tydeni.csv","CZ_RADIO_TOP_50_CZ_tydeni.csv"]

In [140]:
df = pd.DataFrame()
for f in data:
    df = pd.concat([df,pd.read_csv(os.path.join("data",f))])

In [141]:
interpreti = df.groupby("interpret_hlavni")['rok'].nunique().nlargest(5000).index.to_list()

In [142]:
len(interpreti)

997

## Nejdřív si obstaráme tvůrců

In [143]:
import requests

### Zkusíme to automaticky

In [144]:
def get_id(query):
    API_ENDPOINT = "https://www.wikidata.org/w/api.php"
    params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'language': 'en',
        'search': query
    }

    r = requests.get(API_ENDPOINT, params = params)
    
    try:
        
        return[r.json()['search'][0]['id'], r.json()['search'][0]['display']['label']['value'], r.json()['search'][0]['display']['description']['value']]
    
    except:
        return[None, None, None]

In [145]:
get_id('Chainsmokers')

[None, None, None]

In [133]:
from IPython.display import clear_output

In [146]:
qecka = {}
count = 0
for i in interpreti:
    print(f"""{count} / {len(interpreti)} / {i}""")
    qecka[i] = get_id(i)
    count = count + 1
    clear_output(wait=True)

996 / 997 / - Warner Music


In [136]:
qecka

{'The Weeknd': ['Q2121062',
  'The Weeknd',
  'Canadian singer-songwriter (born 1990)'],
 'Ariana Grande': ['Q151892',
  'Ariana Grande',
  'American singer and actress (born 1993)'],
 'Mariah Carey': ['Q41076',
  'Mariah Carey',
  'American singer and songwriter (born 1969)'],
 'David Guetta': ['Q8298', 'David Guetta', 'French DJ and record producer'],
 'Calvin Harris': ['Q81637',
  'Calvin Harris',
  'British DJ, singer and record producer'],
 'Sia': ['Q181484', 'Sia', 'Australian singer and songwriter'],
 'Selena Gomez': ['Q83287',
  'Selena Gomez',
  'American singer and actress (born 1992)'],
 'Wham!': ['Q744087', 'Wham!', 'English pop duo'],
 'Yzomandias': ['Q29066314', 'Yzomandias', 'Czech rapper'],
 'Maroon 5': ['Q182223', 'Maroon 5', 'American pop rock band'],
 'Ed Sheeran': ['Q47447', 'Ed Sheeran', 'English singer-songwriter'],
 'Imagine Dragons': ['Q391348', 'Imagine Dragons', 'American pop rock band'],
 'Calin': ['Q64785850', 'George A. Calin', 'Molecular Pathology research

In [147]:
if not os.path.exists("wikidata"):
    os.mkdir("wikidata")
else:
    pass

In [ ]:
with open(os.path.join("wikidata","wiki_raw.json"), "w+") as file:
    json.dump(h, file)

### Ručo doplnění kódů

## Wikidata: the real shit

Chcu:
- žánr
- kapela/jednotlivec? (Ale bacha u interpret_hlavni!)
- země původu (co když víc?)
- rok narození
- místo narození
- rok úmrtí
- místo úmrtí
- rok začátku kariéry
Možná ještě:
- ocenění?

In [18]:
from wikidata.client import Client
import wikidata.entity

In [72]:
def wikidates(id):
    client = Client()
    entity = client.get(id, load=True)
    return entity

### Začátek a konec kariéry

In [95]:
wikidates('Q45188').attributes['claims']['P2031'][0]['mainsnak']['datavalue']['value']['time']

'+1997-00-00T00:00:00Z'

In [96]:
wikidates('Q45188').attributes['claims']['P2032'][0]['mainsnak']['datavalue']['value']['time']

KeyError: 'P2032'

work period (end) (P2032)

pohlavní identita subjektu; u lidí použijte muž (Q6581097), žena (Q6581072), transmuž (Q2449503), transžena (Q1052281); u zvířat použijte samec (Q44148), samice (Q43445); u skupin stejného pohlaví použijte „podtřída (čeho)“ (P279)

In [73]:
wikidates('Q45188').attributes['claims']

{'P358': [{'mainsnak': {'snaktype': 'value',
    'property': 'P358',
    'hash': '4cd91f2104436f2b24b0740042ca699db2f1cfc6',
    'datavalue': {'value': {'entity-type': 'item',
      'numeric-id': 547583,
      'id': 'Q547583'},
     'type': 'wikibase-entityid'},
    'datatype': 'wikibase-item'},
   'type': 'statement',
   'id': 'q45188$BD0D253C-AF10-4F3B-9671-238F964C8D08',
   'rank': 'normal',
   'references': [{'hash': 'fa278ebfc458360e5aed63d5058cca83c46134f1',
     'snaks': {'P143': [{'snaktype': 'value',
        'property': 'P143',
        'hash': 'e4f6d9441d0600513c4533c672b5ab472dc73694',
        'datavalue': {'value': {'entity-type': 'item',
          'numeric-id': 328,
          'id': 'Q328'},
         'type': 'wikibase-entityid'},
        'datatype': 'wikibase-item'}]},
     'snaks-order': ['P143']}]}],
 'P373': [{'mainsnak': {'snaktype': 'value',
    'property': 'P373',
    'hash': 'c93465ee1cdfb565aa39c69a7275c57b1069cef1',
    'datavalue': {'value': 'Coldplay', 'type': 'st

### Země původu

In [81]:
wikidates('Q45188').attributes['claims']['P495'][0]['mainsnak']['datavalue']['value']['id']

'Q145'

In [88]:
wikidates('Q145').attributes['claims']['P1705'][0]['mainsnak']['datavalue']['value']['text']

'United Kingdom of Great Britain and Northern Ireland'

In [42]:
def wikidates2(id):
    client = Client()
    entity = client.get(id, load=True)
    return dict(wikidata.entity.Entity("P551", client))

In [43]:
wikidates2('Q33240')

{<wikidata.entity.Entity P31>: <wikidata.entity.Entity Q18608871>,
 <wikidata.entity.Entity P1659>: <wikidata.entity.Entity P937>,
 <wikidata.entity.Entity P1855>: <wikidata.entity.Entity Q60543>,
 <wikidata.entity.Entity P2875>: <wikidata.entity.Entity Q23909040>,
 <wikidata.entity.Entity P1628>: 'https://schema.org/homeLocation',
 <wikidata.entity.Entity P3254>: 'https://www.wikidata.org/wiki/Wikidata:Property_proposal/Archive/7#P551',
 <wikidata.entity.Entity P2302>: <wikidata.entity.Entity Q21510865>,
 <wikidata.entity.Entity P1647>: <wikidata.entity.Entity P276>,
 <wikidata.entity.Entity P1629>: <wikidata.entity.Entity Q699405>,
 <wikidata.entity.Entity P3734>: <wikidata.entity.Entity Q42533339>,
 <wikidata.entity.Entity P6609>: <wikidata.entity.Entity P131>,
 <wikidata.entity.Entity P8274>: <wikidata.entity.Entity Q44601380>,
 <wikidata.entity.Entity P7087>: <wikidata.entity.Entity Q107616534>,
 <wikidata.entity.Entity P2668>: <wikidata.entity.Entity Q24025284>}

In [65]:
def get_gender(wikidata_id):
    client = Client()  
    
    entity = client.get(wikidata_id)  # Retrieve the entity using the Wikidata ID
        
    print(entity.attributes['claims']['P21'][0]['mainsnak']['datavalue']['value']['id'])

In [67]:
get_gender('Q45188')

KeyError: 'P21'